In [1]:
import soundfile as sf
import librosa
import numpy as np
import matplotlib.pyplot as plt
import scipy.fft as sp
from numpy import fft

In [2]:
# Take in data path
# Return [spoof list, bonafide list]
def create_bf_spf_lists(path):
    f = open(path)
    spoof_list = []
    bonafide_list = []
    for line in f:
        tokens = line.strip().split()
        if tokens[3] == 'spoof':
            spoof_list.append(tokens[1])
        else:
            bonafide_list.append(tokens[1])
    return [spoof_list, bonafide_list]
'''
[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.female.trl.txt")
[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.dev.male.trl.txt")
'''

[spoof_listf, bonafide_listf] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.female.trl.txt")
[spoof_listm, bonafide_listm] = create_bf_spf_lists("DS_10283_3336\LA\LA\ASVspoof2019_LA_asv_protocols\ASVspoof2019.LA.asv.eval.male.trl.txt")

In [3]:
def extract_time_data(list, num):
    time_data = []
    #fspoof = open(path, "w")
    for i in range(num):
        #dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_dev\\flac\\' + list[i] + '.flac'
        dpath = 'DS_10283_3336\LA\LA\ASVspoof2019_LA_eval\\flac\\' + list[i] + '.flac'
        
        x, sr = librosa.load(dpath, sr=44100)
        time_data.append(x)
    return (time_data, sr)

ftime_data_spoof, sr = extract_time_data(spoof_listf, 4500)
ftime_data_bonafide, sr = extract_time_data(bonafide_listf, 1500)
mtime_data_spoof, sr = extract_time_data(spoof_listm, 4500)
mtime_data_bonafide, sr = extract_time_data(bonafide_listm, 1500)

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import librosa


def lin(sr, n_fft, n_filter=128, fmin=0.0, fmax=None, dtype=np.float32):

    if fmax is None:
        fmax = float(sr) / 2
    # Initialize the weights
    n_filter = int(n_filter)
    weights = np.zeros((n_filter, int(1 + n_fft // 2)), dtype=dtype)

    # Center freqs of each FFT bin
    fftfreqs = librosa.fft_frequencies(sr=sr, n_fft=n_fft)

    # 'Center freqs' of liner bands - uniformly spaced between limits
    linear_f = np.linspace(fmin, fmax, n_filter + 2)

    fdiff = np.diff(linear_f)
    ramps = np.subtract.outer(linear_f, fftfreqs)

    for i in range(n_filter):
        # lower and upper slopes for all bins
        lower = -ramps[i] / fdiff[i]
        upper = ramps[i + 2] / fdiff[i + 1]

        # .. then intersect them with each other and zero
        weights[i] = np.maximum(0, np.minimum(lower, upper))

    return weights


def linear_spec(y=None,
                sr=22050,
                n_fft=2048,
                hop_length=16384,
                win_length=1024,
                window='hann',
                center=True,
                pad_mode='reflect',
                power=2.0,
                **kwargs):
    S = np.abs(
        librosa.core.stft(y=y,
                          n_fft=n_fft,
                          hop_length=hop_length,
                          win_length=win_length,
                          window=window,
                          center=center,
                          pad_mode=pad_mode))**power
    filter = lin(sr=sr, n_fft=n_fft, **kwargs)
    return np.dot(filter, S)


def lfcc(y=None,
         sr=22050,
         S=None,
         n_lfcc=20,
         dct_type=2,
         norm='ortho',
         **kwargs):
    if S is None:
        S = librosa.power_to_db(linear_spec(y=y, sr=sr, **kwargs))
    M = scipy.fftpack.dct(S, axis=0, type=dct_type, norm=norm)[:n_lfcc]
    return M

In [ ]:
def get_max_length(data):
    return max(len(vector) for vector in data)

def pad_or_truncate(data, max_length):
    padded_data = []
    for vector in data:
        if len(vector) < max_length:
            pad_width = max_length - len(vector)
            padded_vector = np.pad(vector, pad_width=(0, pad_width), mode='constant')
        elif len(vector) > max_length:
            padded_vector = vector[:max_length]
        else:
            padded_vector = vector
        padded_data.append(padded_vector)
    return padded_data

# Extract mfccs from time_data with padding
def extract_lfccs(time_data, max_length, sr=44100, n_lfcc=20):
    padded_data = pad_or_truncate(time_data, max_length)
    
    lfcc_data = []
    
    #for x in padded_data:
    for x in time_data:
        lfcc_r = lfcc(y=x, sr=44100, S=None, n_lfcc=20, dct_type=2, norm='ortho')
        # Flatten to a large vector

        flattened_vector = np.array(lfcc_r).flatten()
        lfcc_data.append(flattened_vector)
        print(lfcc_r.shape)
        
        #lfcc_data.append(lfcc_r)
    
    return np.array(lfcc_data)

fmax_length = max(get_max_length(ftime_data_spoof), get_max_length(ftime_data_bonafide))
mmax_length = max(get_max_length(mtime_data_spoof), get_max_length(mtime_data_bonafide))
max_length = max(fmax_length, mmax_length)

lfccs_spooff = extract_lfccs(ftime_data_spoof, max_length, sr=44100, n_lfcc=20)
lfccs_bonafidef = extract_lfccs(ftime_data_bonafide, max_length, sr=44100, n_lfcc=20)

lfccs_spoofm = extract_lfccs(mtime_data_spoof, max_length, sr=44100, n_lfcc=20)
lfccs_bonafidem = extract_lfccs(mtime_data_bonafide, max_length, sr=44100, n_lfcc=20)

(20, 7)
(20, 7)
(20, 7)
(20, 7)
(20, 6)
(20, 7)
(20, 7)
(20, 6)
(20, 6)
(20, 6)
(20, 4)
(20, 5)
(20, 5)
(20, 4)
(20, 4)
(20, 4)
(20, 5)
(20, 4)
(20, 4)
(20, 4)
(20, 7)
(20, 9)
(20, 9)
(20, 8)
(20, 9)
(20, 8)
(20, 9)
(20, 8)
(20, 8)
(20, 7)
(20, 3)
(20, 5)
(20, 5)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 3)
(20, 16)
(20, 18)
(20, 18)
(20, 16)
(20, 16)
(20, 16)
(20, 17)
(20, 15)
(20, 15)
(20, 16)
(20, 7)
(20, 8)
(20, 8)
(20, 8)
(20, 7)
(20, 7)
(20, 7)
(20, 7)
(20, 7)
(20, 7)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 3)
(20, 3)
(20, 3)
(20, 8)
(20, 9)
(20, 9)
(20, 9)
(20, 8)
(20, 8)
(20, 9)
(20, 8)
(20, 9)
(20, 8)
(20, 5)
(20, 7)
(20, 7)
(20, 6)
(20, 6)
(20, 6)
(20, 6)
(20, 5)
(20, 5)
(20, 5)
(20, 5)
(20, 6)
(20, 6)
(20, 7)
(20, 5)
(20, 6)
(20, 6)
(20, 6)
(20, 6)
(20, 5)
(20, 4)
(20, 5)
(20, 5)
(20, 5)
(20, 4)
(20, 4)
(20, 5)
(20, 4)
(20, 4)
(20, 4)
(20, 3)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 4)
(20, 3)
(20, 4)
(20, 3)
(20, 8)
(20, 10)
(20, 10)
(20,

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4500,) + inhomogeneous part.

In [ ]:
np.set_printoptions(threshold=np.inf, precision=6, linewidth=np.inf, suppress=True)

# Open files for writing
with open("spoof_data_lfccs1_male.txt", "w") as fspoofw:
    for row in lfccs_spoofm:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_data_lfccs1_male.txt", "w") as fbonafidew:
    for row in lfccs_bonafidem:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')

with open("spoof_data_lfccs1_female.txt", "w") as fspoofw:
    for row in lfccs_spooff:
        array_str = np.array_str(row)
        fspoofw.write(array_str + '\n')

with open("bonafide_data_lfccs1_female.txt", "w") as fbonafidew:
    for row in lfccs_bonafidef:
        array_str = np.array_str(row)
        fbonafidew.write(array_str + '\n')